In [2]:
import pandas as pd
import numpy as np

In [3]:
with open('../../inputs/0079_keylog.txt', 'r') as f:
    keylog_string = f.read()

I thought of this problem as a directed graph/tree based problem. In particular, I saw this problem as an application of the [toposort][1] (topological sorting) algorithm. Below is a general reasoning for how to apply it to this problem.

Define each vertex as a digit. For each keylog, we create 3 directed edges: from the first to second number, from the first to third number, and finally from the second to third number (i.e., a directed edge between $A$ and $B$ tells us at least one keylog had $A$ before $B$). If there are cycles, then we have more work to do, since toposort is not possible with cycles. Fortunately, in this set of keylogs, there were no cycles (i.e., if $A$ is before $B$ then we know $B$ is not before $A$, as well).

The passcode generation is simply toposort from there (I personally use Kahn's algorithm, although depth-first search would work well here, too). The only caveat here is that there are nodes here with no in-edges and no out-edges, so be sure to not include those in the final passcode--in this case, $4$ and $5$ should not be in the passcode.

We want to start with the number $N_0$ that has in-degree $0$ (and out-degree $> 0$), since this would suggest nothing can come before $N_0$. Append that number to the passcode. Then, for any number that has a directed edge from $N_0$ to them, we simply subtract $1$ from their in-degree. Then we repeat, looking for in-degree $0$, appending to the passcode, then subtracting $1$ from future in-degrees.

In this particular problem, every number was already in order by in-degree, and so we could just read off the passcode in order, instead of implementing Kahn's algorithm:
* 7 comes first (in-degree = 0)
* 3 comes second (in-degree = 1)
* 1 comes third (in-degree = 2)

$\quad\quad\quad \vdots$

  [1]: https://en.wikipedia.org/wiki/Topological_sorting

In [4]:
keylogs = [[int(c) for c in n] for n in keylog_string.split('\n')]

# store edge information
edges = np.zeros((10,10), int)

# if edges[b, a] = 1, this means b has an in-edge from a
# in the passcode, this means b comes after a
for keylog in keylogs:
    edges[keylog[0], keylog[1]] = 1
    edges[keylog[0], keylog[2]] = 1
    edges[keylog[1], keylog[2]] = 1

# # we can use this to see if there are any numbers that are in cycles (i.e., a before b and b before a)
# # in this case there are not
# for i in range(10):
#     for j in range(10):
#         if edges[i,j] and edges[j,i]:
#             print(i,j)

s1, s2 = np.sum(edges, axis = 0), np.sum(edges, axis = 1)
passcode = np.zeros(10)
for i,j in enumerate(s1):
    # if a number has no in-edges or out-edges, then it doesn't need to be in the passcode
    if s1[i] == s2[i] == 0:
        passcode = passcode[:-1]
        continue

    passcode[j] = i


print(passcode)


[7. 3. 1. 6. 2. 8. 9. 0.]
